In [31]:
import pandas as pd 
import numpy as np
from nilearn import input_data
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
import glob
from tqdm import tqdm
from nilearn.connectome import ConnectivityMeasure
import os
import nibabel as nib
import nilearn

# Extract Netwokrs in AAL atlas based on yeo7

In [172]:
aal_atlas = datasets.fetch_atlas_aal(version='SPM12')
aal_maps = aal_atlas.maps

atlas_yeo_2011 = datasets.fetch_atlas_yeo_2011()
atlas_yeo = atlas_yeo_2011.thick_7


In [173]:
regions = np.array(aal_atlas['labels'])



In [174]:
yeo_re_file = nilearn.image.resample_to_img(nib.load(atlas_yeo), nib.load(aal_maps))

In [175]:
yeo_re = np.squeeze(yeo_re_file.get_fdata())
aal =  nib.load(aal_maps).get_fdata()

In [176]:
aal_ind_w = np.array(aal_atlas.indices).astype(np.int64)
aal_ind_n = np.arange(0,116)


In [177]:
from numpy import copy
d = dict(zip(aal_ind_w,aal_ind_n))
d[0] = -1
aal_corr = copy(aal)
for k, v in d.items(): aal_corr[aal==k] = v

In [310]:
yeo_re = yeo_re.astype(np.int64)
np.unique(yeo_re)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [211]:
aal_networks = []

for i in np.unique(yeo_re):
    network_ind = np.where(yeo_re==i)
    aal_network_i = aal_corr[network_ind]
    #Find overlpping rois with vol > 0.5 orig size
    rois = []
    for roi in np.unique(aal_network_i):
        roi_size = np.count_nonzero(aal_corr == roi)
        roi_size_network = np.count_nonzero(aal_network_i == roi)
        if roi_size_network >= 0.2 * roi_size:
            rois.append(int(roi))
    aal_networks.append(rois)
    

In [212]:
network_names = ['Visual', 'Somatomotor', 'Dorsal Attention', 'Ventral Attention', 'Limbic', 'Frotoparital', 'Default']
for i, n in enumerate(aal_networks[1:]):
    print(f'{network_names[i]} Network : {len(n)} regions')
    print(regions[n])
    print('*****************************')

Visual Network : 15 regions
['ParaHippocampal_R' 'Calcarine_L' 'Calcarine_R' 'Cuneus_L' 'Cuneus_R'
 'Lingual_L' 'Lingual_R' 'Occipital_Sup_L' 'Occipital_Sup_R'
 'Occipital_Mid_L' 'Occipital_Mid_R' 'Occipital_Inf_L' 'Occipital_Inf_R'
 'Fusiform_L' 'Fusiform_R']
*****************************
Somatomotor Network : 15 regions
['Precentral_L' 'Precentral_R' 'Rolandic_Oper_L' 'Rolandic_Oper_R'
 'Supp_Motor_Area_L' 'Supp_Motor_Area_R' 'Insula_R' 'Postcentral_L'
 'Postcentral_R' 'Paracentral_Lobule_L' 'Paracentral_Lobule_R' 'Heschl_L'
 'Heschl_R' 'Temporal_Sup_L' 'Temporal_Sup_R']
*****************************
Dorsal Attention Network : 9 regions
['Precentral_L' 'Parietal_Sup_L' 'Parietal_Sup_R' 'Parietal_Inf_L'
 'Parietal_Inf_R' 'Precuneus_L' 'Precuneus_R' 'Temporal_Inf_L'
 'Temporal_Inf_R']
*****************************
Ventral Attention Network : 12 regions
['Frontal_Inf_Oper_L' 'Frontal_Inf_Oper_R' 'Rolandic_Oper_L'
 'Rolandic_Oper_R' 'Supp_Motor_Area_L' 'Supp_Motor_Area_R' 'Insula_L'
 'In

In [156]:
flat_list = [item for sublist in aal_networks for item in sublist]

In [222]:
np.save("networks.npy",dict(zip(network_names,aal_networks[1:])))
np.load('networks.npy', allow_pickle=True).item()['Visual']

[39, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55]

In [292]:
dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm', symmetric_split= True)
ho_cor = dataset.maps
labels_cor = dataset.labels[1:]

dataset = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm', symmetric_split= True)
ho_sub = dataset.maps
labels_sub = dataset.labels[1:]

In [293]:
regions_ho = labels_cor + labels_sub

In [311]:
ho_c =  ho_cor.get_fdata()
ho_s = ho_sub.get_fdata()

In [309]:
yeo_re_file = nilearn.image.resample_to_img(nib.load(atlas_yeo), ho_cor)

In [314]:
ho_c = ho_c -1 

In [296]:
sub_ind = np.arange(97,119)
d = dict(zip(np.arange(1,24) ,sub_ind))
ho_s_corr = copy(ho_s)
for k, v in d.items(): ho_s_corr[ho_s==k] = v

In [399]:
ho_networks = []

for i in np.unique(yeo_re):
    network_ind = np.where(yeo_re==i)
    ho_network_i = ho_c[network_ind]
    #Find overlpping rois with vol > 0.5 orig size
    rois = []
    for roi in np.unique(ho_network_i):
        roi_size = np.count_nonzero(ho_c == roi)
        roi_size_network = np.count_nonzero(ho_network_i == roi)
        if roi_size_network >= 0.5 * roi_size:
            rois.append(int(roi))
    ho_networks.append(rois)
    

In [400]:
network_names = ['Visual', 'Somatomotor', 'Dorsal Attention', 'Ventral Attention', 'Limbic', 'Frotoparital', 'Default']
for i, n in enumerate(ho_networks[1:]):
    print(f'{network_names[i]} Network : {len(n)} regions')
    print(np.array(labels_cor)[n])
    print('*****************************')

Visual Network : 17 regions
['Left Lateral Occipital Cortex, inferior division'
 'Right Lateral Occipital Cortex, inferior division'
 'Left Intracalcarine Cortex' 'Right Intracalcarine Cortex'
 'Left Cuneal Cortex' 'Right Cuneal Cortex'
 'Left Parahippocampal Gyrus, posterior division' 'Left Lingual Gyrus'
 'Right Lingual Gyrus' 'Left Temporal Occipital Fusiform Cortex'
 'Right Temporal Occipital Fusiform Cortex'
 'Left Occipital Fusiform Gyrus' 'Right Occipital Fusiform Gyrus'
 'Left Supracalcarine Cortex' 'Right Supracalcarine Cortex'
 'Left Occipital Pole' 'Right Occipital Pole']
*****************************
Somatomotor Network : 14 regions
['Left Precentral Gyrus' 'Right Precentral Gyrus'
 'Left Superior Temporal Gyrus, posterior division'
 'Left Postcentral Gyrus' 'Right Postcentral Gyrus'
 'Left Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex)'
 'Left Central Opercular Cortex' 'Right Central Opercular Cortex'
 'Left Parietal Operculum Cortex' 'Right Parietal O

In [364]:
ho_networks.append(np.arange(96,96+22).tolist())

In [365]:
network_names = ['Visual', 'Somatomotor', 'Dorsal Attention', 'Ventral Attention', 'Limbic', 'Frotoparital', 'Default','Subcortical']
np.save("ho_networks.npy",dict(zip(network_names,ho_networks[1:])))



In [402]:
network_names

['Visual',
 'Somatomotor',
 'Dorsal Attention',
 'Ventral Attention',
 'Limbic',
 'Frotoparital',
 'Default']

In [401]:
ho_networks

[[-1],
 [44, 45, 46, 47, 62, 63, 68, 70, 71, 76, 77, 78, 79, 92, 93, 94, 95],
 [12, 13, 18, 32, 33, 50, 82, 83, 84, 85, 88, 89, 90, 91],
 [24, 30, 34],
 [2, 3, 80, 81],
 [14, 15, 26, 27, 29, 48, 49, 52, 53, 66, 67, 72, 73, 75],
 [6],
 [5, 9, 11, 16, 17, 19, 20, 21, 22, 23, 40, 41, 54, 55, 59, 69]]

In [419]:
import torch
x = torch.ones((3,100,4))
m = torch.tensor([0,0,1,1])
mask = torch.where(m==1)[0]
z = x[:,:,mask]
z.shape


torch.Size([3, 100, 2])

In [425]:
f = z - z.mean(0,keepdim=True)
f.shape

torch.Size([3, 100, 2])

In [381]:
dataset = nilearn.datasets.fetch_coords_dosenbach_2010()

/data/agelgazzar/anaconda3/envs/ssm/lib/python3.9/site-packages/nilearn/datasets/atlas.py:1261: UserWarning: `legacy_format` will default to `False` in release 0.11. Dataset fetchers will then return pandas dataframes by default instead of recarrays.
  warnings.warn(_LEGACY_FORMAT_MSG)


In [370]:
np.where(dataset['labels'] == 'thalamus 57')

array(['inf cerebellum 155', 'inf cerebellum 150', 'inf cerebellum 151',
       'inf cerebellum 140', 'inf cerebellum 131', 'inf cerebellum 122',
       'inf cerebellum 121', 'inf cerebellum 110', 'lat cerebellum 128',
       'lat cerebellum 113', 'lat cerebellum 109', 'lat cerebellum 98',
       'med cerebellum 143', 'med cerebellum 144', 'med cerebellum 138',
       'med cerebellum 130', 'med cerebellum 127', 'med cerebellum 120',
       'ACC 19', 'TPJ 125', 'aPFC 8', 'angular gyrus 102',
       'ant insula 28', 'ant insula 26', 'basal ganglia 71',
       'basal ganglia 38', 'basal ganglia 39', 'basal ganglia 30',
       'dACC 27', 'fusiform 81', 'mFC 31', 'mid insula 61',
       'mid insula 59', 'mid insula 44', 'parietal 97', 'parietal 89',
       'post cingulate 80', 'post insula 76', 'precuneus 87',
       'sup temporal 100', 'temporal 103', 'temporal 95', 'temporal 78',
       'thalamus 57', 'thalamus 58', 'thalamus 47', 'vFC 40', 'vFC 33',
       'vFC 25', 'vPFC 18', 'ACC 14', 

In [377]:
np.where(dataset['labels'] == 'thalamus 57')

(array([43]),)

In [382]:
networks = dataset['networks'].unique()

In [394]:
np.where(dataset['networks']=='sensorimotor')

(array([127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
        153, 154, 155, 156, 157, 158, 159]),)

In [391]:
dos_networks = []
    
for network in networks:
    network_ind = np.where(dataset['networks']==network)[0].tolist()
    dos_networks.append(network_ind)
    

In [385]:
networks_list = networks.tolist()

In [393]:
np.save("dos_networks.npy",dict(zip(networks_list,dos_networks)))

In [387]:
networks_list

['cerebellum',
 'cingulo-opercular',
 'default',
 'fronto-parietal',
 'occipital',
 'sensorimotor']

In [392]:
dos_networks

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 [18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49],
 [50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83],
 [84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104],
 [105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126],
 [127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159]]

In [389]:
ho_networks

[[-1],
 [44, 45, 46, 47, 62, 63, 68, 70, 71, 76, 77, 78, 79, 92, 93, 94, 95],
 [12, 13, 18, 32, 33, 82, 83, 88, 89, 90, 91],
 [24, 30, 34],
 [2, 3, 80, 81],
 [14, 26, 27, 49, 52, 53, 66, 67, 72, 73, 75],
 [6],
 [9, 16, 17, 20, 21, 22, 23, 40, 41, 55, 59, 69],
 [96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117]]